In [3]:
from ket import *
from ket.lib import *
from math import log2, gcd, pi
from random import randint

from typing import Literal

In [5]:
def print_state(
    *qubits, format: list[Literal["int", "bin"]] | Literal["int", "bin"] | None = None
):
    """
    Imprime o estado do sistema quântico.

    Args:
        *qubits (Quant): Os qubits cujos estados serão impressos.
        format (list[str] | str | None): O formato de impressão para cada qubit. Pode ser uma lista contendo
            "int" ou "bin" para cada qubit, uma string "int" ou "bin" que será aplicada a todos os qubits,
            ou None para usar o formato padrão "bin".

    Raises:
        AssertionError: Se o argumento `format` for uma string diferente de "int" ou "bin", ou se o tamanho da
            lista `format` não for igual ao número de qubits.
    """
    from itertools import accumulate
    from functools import reduce
    from operator import add
    import ket
    from IPython.display import Math, display

    if isinstance(format, str):
        assert format in ["int", "bin"], "format deve ser 'int' ou 'bin'"
        format = [format] * len(qubits)
    elif format is not None:
        assert len(format) == len(qubits), "format deve ter o mesmo tamanho que qubits"
        assert all(
            f in ["int", "bin"] for f in format
        ), "format deve conter apenas 'int' ou 'bin'"
    else:
        format = ["bin"] * len(qubits)

    qubit_len = list(map(len, qubits))
    split_indices = [0] + list(accumulate(qubit_len))

    qubits = reduce(add, qubits)
    qubits = ket.dump(qubits)
    math = []
    for state, amp in qubits.get().items():

        def float_to_math(num: float, is_complex: bool) -> str | None:
            num_str = None
            if abs(num) > 1e-14:

                sqrt_dem_float = 1 / num**2
                sqrt_dem = round(sqrt_dem_float)
                if abs(sqrt_dem - sqrt_dem_float) < 1e-10 and sqrt_dem != 1:
                    num_str = f"\\frac{{{'-' if num < 0.0 else ''}{'i' if is_complex else '1'}}}{{\\sqrt{{{sqrt_dem}}}}}"
                else:
                    round_num = round(num)
                    if abs(round_num - num) > 1e-14:
                        num_str = str(num)
                        if "e" in num_str:
                            num_str = num_str.replace("e", "\\times10^{") + "}"
                    else:
                        num_str = ""
                    if is_complex:
                        num_str += "i"
            return num_str

        real_str = float_to_math(amp.real, False)
        imag_str = float_to_math(amp.imag, True)

        state_str = f"{state:0{len(qubits.qubits)}b}"
        state_str = [
            f"\\left|{state_str[start:end] if fmt == 'bin' else int(state_str[start:end], 2)}\\right>"
            for start, end, fmt in zip(split_indices, split_indices[1:], format)
        ]
        state_str = "".join(state_str)

        if real_str is not None and imag_str is not None:
            math.append(f"({real_str}+{imag_str}i) {state_str}")
        else:
            math.append(f"{real_str if real_str is not None else imag_str} {state_str}")

    display(Math("+".join(math).replace("+-", "-")))

In [16]:
def qft(qubits: Quant, invert: bool = True):
    if len(qubits) == 1:
        H(qubits)
    else:
        *head, tail = qubits
        H(tail)
        for i, ctrl_qubit in enumerate(reversed(head)):
            ctrl(ctrl_qubit, PHASE(pi / 2 ** (i + 1)))(tail)
        qft(head, invert=False)
    if invert:
        size = len(qubits)
        for i in range(size // 2):
            SWAP(qubits[i], qubits[size - i - 1])

In [6]:
N = 15
n = N.bit_length()

In [14]:
# Processo Ket
p = Process()

# Alocação qubits de controle e alvo
control_qubits = p.alloc(2*n)
target_qubits = p.alloc(n)
print_state(control_qubits)
print_state(target_qubits)

# Superposição dos qubits de controle
H(control_qubits)
# print_state(control_qubits)

# Inicialização do qubit alvo em \ket{1}
X(target_qubits[-1])
print_state(target_qubits)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [15]:
medida = sample(control_qubits)
print(medida.value)

{226: 7, 228: 10, 118: 9, 146: 11, 154: 6, 236: 7, 251: 8, 36: 5, 42: 3, 199: 5, 77: 12, 169: 12, 223: 6, 84: 10, 242: 10, 28: 7, 188: 5, 151: 9, 119: 11, 80: 8, 35: 8, 163: 12, 105: 10, 53: 4, 160: 8, 26: 7, 134: 5, 141: 11, 219: 11, 92: 6, 139: 6, 253: 10, 234: 10, 216: 8, 241: 8, 246: 7, 180: 9, 229: 6, 69: 8, 0: 10, 65: 11, 181: 10, 49: 5, 91: 12, 222: 10, 122: 11, 166: 4, 27: 3, 195: 6, 155: 5, 211: 8, 174: 9, 67: 10, 7: 7, 186: 8, 81: 11, 214: 6, 120: 4, 115: 9, 83: 8, 217: 10, 29: 9, 212: 4, 8: 9, 124: 1, 254: 13, 191: 9, 202: 5, 203: 8, 200: 11, 57: 7, 248: 9, 149: 7, 110: 9, 76: 9, 230: 7, 187: 11, 15: 3, 3: 11, 239: 4, 111: 5, 102: 8, 143: 9, 11: 7, 224: 7, 78: 5, 245: 5, 207: 6, 109: 9, 16: 11, 172: 9, 152: 11, 127: 13, 179: 14, 39: 13, 153: 8, 235: 7, 194: 9, 158: 7, 192: 7, 97: 12, 22: 10, 60: 11, 227: 4, 135: 6, 100: 6, 185: 7, 51: 6, 68: 6, 59: 4, 221: 5, 70: 1, 140: 14, 123: 12, 90: 6, 33: 4, 205: 8, 43: 6, 126: 4, 23: 9, 250: 5, 40: 8, 66: 4, 1: 4, 71: 10, 209: 5, 72: 

In [ ]:
# target_qubits.